In [1]:
# Basic set up
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/itc-frank/Panama imp 2019.csv


## Context: 

The Excel file Panama imp 2019.xlsx, presents Panama’s imports quantitative information (values and quantities) for the year 2019 at different level of aggregation: Total, by country, by country and product.

The goal of the exercise is to re-structure the information and keep only quantitative information reported by product and country.

Prepare a txt file with the different variables:

- Flow
- Period of time (year 2019)
- Product code (numeral) "Code"
- Partner country ("country")
- Value ("CIF")
- Quantity (net weight) ("net")
The result file must look like this example: 



In [2]:
df = pd.read_csv('/kaggle/input/itc-frank/Panama imp 2019.csv')
df.head(20)

,Code,Unnamed: 1,net,CIF,tax,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,Albania,"3,465","85,558","2,428",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0910.99.90.00,Otras especies.,93,"1,398",138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3926.90.49.00,"Objetos de laboratorio y farmacia, incluso...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"calibrados, de plástico.",5,411,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4819.40.19.00,"Sacos (bolsas), multipliegos, de papel, cartón...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,"celulosa o napa de fibras de celulosa, exc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,"cemento, café molido o azúcar.",98,880,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6104.43.10.00,"Vestidos de punto, para mujeres, de fibras sin...",2,112,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6104.69.00.00,"Pantalones largos o cortos (calzones), con pet...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,"de punto, para mujeres o niñas, de las dem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Problems discovered:
1. Multiple rows should be merged into single records. (rows 2,3 should be merged, and rows 4, 5, 6 should be merged)
2. The country rows (ex. row 0) should be shaped into columns. 

In this step, we only care about columns: Code, net, CIF.

In [3]:
#Drop unamed columns
df= df.dropna(axis=1, how='all')

## Pattern discovered: 
For each chunck of rows that are supposed to be merged into one record (here refer such chunck of rows as record): For Code column, valid info appears in the first row; for "net" and "CIF" columns, valide info apperas in the last row. 

For example, rows 4,5,6 are supposed to be merged into one record. The valid info for Code column appears in row 4, and the valid info for "net" and "CIF" columns appear in row 6.

Given this pattern, I use forward fill `.ffill()` and backword fill `.bfill()` to replace the `NaN` value with either the first row or last row per record. 


In [4]:
df["Code"]=df["Code"].ffill()
df["net"]=df["net"].bfill()
df["CIF"]=df["CIF"].bfill()
#Let's look at the result
df.head(10)

,Code,Unnamed: 1,net,CIF,tax
0,NaN,Albania,"3,465","85,558","2,428"
1,0910.99.90.00,Otras especies.,93,"1,398",138
2,3926.90.49.00,"Objetos de laboratorio y farmacia, incluso...",5,411,NaN
3,3926.90.49.00,"calibrados, de plástico.",5,411,12
4,4819.40.19.00,"Sacos (bolsas), multipliegos, de papel, cartón...",98,880,NaN
5,4819.40.19.00,"celulosa o napa de fibras de celulosa, exc...",98,880,NaN
6,4819.40.19.00,"cemento, café molido o azúcar.",98,880,131
7,6104.43.10.00,"Vestidos de punto, para mujeres, de fibras sin...",2,112,11
8,6104.69.00.00,"Pantalones largos o cortos (calzones), con pet...",125,"5,073",NaN
9,6104.69.00.00,"de punto, para mujeres o niñas, de las dem...",125,"5,073",NaN


## Extract country info 
Extract country info from rows and save it as a column named "country". 
1. Identify which rows don't contain country info: no space, and doesn't end with "."
2. Create a column called "country" = the identified country rows

In [5]:
# Identify which rows contain country info: the rows that don't contain space " " or 
a = df[~(df['Unnamed: 1'].str.contains(" ")|df['Unnamed: 1'].str.endswith("."))]

df["country"]=a["Unnamed: 1"]
df.head(10)



,Code,Unnamed: 1,net,CIF,tax,country
0,NaN,Albania,"3,465","85,558","2,428",Albania
1,0910.99.90.00,Otras especies.,93,"1,398",138,NaN
2,3926.90.49.00,"Objetos de laboratorio y farmacia, incluso...",5,411,NaN,NaN
3,3926.90.49.00,"calibrados, de plástico.",5,411,12,NaN
4,4819.40.19.00,"Sacos (bolsas), multipliegos, de papel, cartón...",98,880,NaN,NaN
5,4819.40.19.00,"celulosa o napa de fibras de celulosa, exc...",98,880,NaN,NaN
6,4819.40.19.00,"cemento, café molido o azúcar.",98,880,131,NaN
7,6104.43.10.00,"Vestidos de punto, para mujeres, de fibras sin...",2,112,11,NaN
8,6104.69.00.00,"Pantalones largos o cortos (calzones), con pet...",125,"5,073",NaN,NaN
9,6104.69.00.00,"de punto, para mujeres o niñas, de las dem...",125,"5,073",NaN,NaN


**Further clean the data by dropping dulicate values in "Code" column.**

In [6]:
df.drop_duplicates('Code', inplace=True)
df.head(10)

,Code,Unnamed: 1,net,CIF,tax,country
0,NaN,Albania,"3,465","85,558","2,428",Albania
1,0910.99.90.00,Otras especies.,93,"1,398",138,NaN
2,3926.90.49.00,"Objetos de laboratorio y farmacia, incluso...",5,411,NaN,NaN
4,4819.40.19.00,"Sacos (bolsas), multipliegos, de papel, cartón...",98,880,NaN,NaN
7,6104.43.10.00,"Vestidos de punto, para mujeres, de fibras sin...",2,112,11,NaN
8,6104.69.00.00,"Pantalones largos o cortos (calzones), con pet...",125,"5,073",NaN,NaN
11,6109.90.00.00,"""T-shirts"" y camisetas interiores, de punto,...",19,"1,508",NaN,NaN
13,6110.20.00.00,"Suéteres (jerseys), ""pullovers"", ""cardigans""...",12,403,NaN,NaN
15,6201.99.00.00,"Anoraks, cazadoras y artículos similares par...",2,174,NaN,NaN
18,6203.43.19.00,"Los demás pantalones cortos (calzones), y ""s...",1,130,NaN,NaN


**Fill all the `NaN` values in column "country" with the first valid country info pre chunck, using `.ffill()` forward fill.**


In [7]:
df["country"] =df["country"].ffill()
df.head(10)

,Code,Unnamed: 1,net,CIF,tax,country
0,NaN,Albania,"3,465","85,558","2,428",Albania
1,0910.99.90.00,Otras especies.,93,"1,398",138,Albania
2,3926.90.49.00,"Objetos de laboratorio y farmacia, incluso...",5,411,NaN,Albania
4,4819.40.19.00,"Sacos (bolsas), multipliegos, de papel, cartón...",98,880,NaN,Albania
7,6104.43.10.00,"Vestidos de punto, para mujeres, de fibras sin...",2,112,11,Albania
8,6104.69.00.00,"Pantalones largos o cortos (calzones), con pet...",125,"5,073",NaN,Albania
11,6109.90.00.00,"""T-shirts"" y camisetas interiores, de punto,...",19,"1,508",NaN,Albania
13,6110.20.00.00,"Suéteres (jerseys), ""pullovers"", ""cardigans""...",12,403,NaN,Albania
15,6201.99.00.00,"Anoraks, cazadoras y artículos similares par...",2,174,NaN,Albania
18,6203.43.19.00,"Los demás pantalones cortos (calzones), y ""s...",1,130,NaN,Albania


**Delete the columns that we don't need anymore**

In [8]:
del df["tax"]
del df["Unnamed: 1"]
df.head(25)

,Code,net,CIF,country
0,NaN,"3,465","85,558",Albania
1,0910.99.90.00,93,"1,398",Albania
2,3926.90.49.00,5,411,Albania
4,4819.40.19.00,98,880,Albania
7,6104.43.10.00,2,112,Albania
8,6104.69.00.00,125,"5,073",Albania
11,6109.90.00.00,19,"1,508",Albania
13,6110.20.00.00,12,403,Albania
15,6201.99.00.00,2,174,Albania
18,6203.43.19.00,1,130,Albania


**Rename columns**

In [9]:
df.rename(columns={'CIF': 'value', 'net': 'quantity', 'country':'partner','Code':'product_code'}, inplace=True)
df.head(25)

,product_code,quantity,value,partner
0,NaN,"3,465","85,558",Albania
1,0910.99.90.00,93,"1,398",Albania
2,3926.90.49.00,5,411,Albania
4,4819.40.19.00,98,880,Albania
7,6104.43.10.00,2,112,Albania
8,6104.69.00.00,125,"5,073",Albania
11,6109.90.00.00,19,"1,508",Albania
13,6110.20.00.00,12,403,Albania
15,6201.99.00.00,2,174,Albania
18,6203.43.19.00,1,130,Albania


**Remove missing values**

In [10]:
df = df[df['product_code'].notna()]
df = df[df['value'].notna()]
df.head(25)

,product_code,quantity,value,partner
1,0910.99.90.00,93,"1,398",Albania
2,3926.90.49.00,5,411,Albania
4,4819.40.19.00,98,880,Albania
7,6104.43.10.00,2,112,Albania
8,6104.69.00.00,125,"5,073",Albania
11,6109.90.00.00,19,"1,508",Albania
13,6110.20.00.00,12,403,Albania
15,6201.99.00.00,2,174,Albania
18,6203.43.19.00,1,130,Albania
20,6204.12.29.00,2,171,Albania


**Transform Quantity and value from string type to numeric type:**

1. Remove the `,` in the string 
2. Transform string to numeric type


In [11]:
df["value"]=df['value'].str.replace(r'\D', '')
df['value'] = pd.to_numeric(df['value'])

df["quantity"]=df['quantity'].str.replace(r'\D', '')
df['quantity'] = pd.to_numeric(df['quantity'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


## Create additional variables 
based on the task description

1. flow variable 
2. year variable


In [12]:
df["flow"]= np.where(df["value"]>0, 1, 2)
df["year"]= 2019
df.head(25)

,product_code,quantity,value,partner,flow,year
1,0910.99.90.00,93,1398,Albania,1,2019
2,3926.90.49.00,5,411,Albania,1,2019
4,4819.40.19.00,98,880,Albania,1,2019
7,6104.43.10.00,2,112,Albania,1,2019
8,6104.69.00.00,125,5073,Albania,1,2019
11,6109.90.00.00,19,1508,Albania,1,2019
13,6110.20.00.00,12,403,Albania,1,2019
15,6201.99.00.00,2,174,Albania,1,2019
18,6203.43.19.00,1,130,Albania,1,2019
20,6204.12.29.00,2,171,Albania,1,2019


## Export results

In [13]:
df = df[["flow", "year", "product_code", "partner", "value", "quantity"]]
df.to_csv('result.txt', header=None, index=None, sep=' ', mode='a')
